## Import Dataset

In [0]:
import pandas as pd

In [2]:
movies = pd.read_csv('https://raw.githubusercontent.com/skwiliam/Simple_KNN_from_Scratch/master/movies.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/skwiliam/Simple_KNN_from_Scratch/master/ratings.csv')

display(movies.head())
display(ratings.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,5.0,847117005
1,1,2,3.0,847642142
2,1,10,3.0,847641896
3,1,32,4.0,847642008
4,1,34,4.0,847641956


## Create Feature

#### Feature for Item Based Recommender System


In [3]:
#merge two dataset on movieId
item_based = pd.merge(movies[['movieId', 'title']], ratings, on='movieId')

#make a pivot dataframe
item_based = item_based.pivot_table(index='title',columns='userId',values='rating').fillna(0).reset_index().rename_axis(None, axis=1)

item_based.head()

,title,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718
0,"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Feature for Content Based Recommender System

Calculate weighted rating each movie

In [4]:
#calculate weighted rating for each movie
content_based = pd.merge(movies[['movieId', 'title', 'genres']], ratings[['movieId', 'userId', 'rating']], on='movieId')
avgRating = content_based.groupby('movieId').rating.mean().to_list()
content_based = content_based.groupby(['movieId', 'title','genres']).rating.count().reset_index(name='numRating')
content_based['avgRating'] = avgRating

def imdb_weighted_rating(df, var=0.8):
    v = df['numRating']
    R = df['avgRating']
    C = df['avgRating'].mean()
    m = df['numRating'].quantile(var)

    df['score'] = (v/(m + v))*R + (m/(m + v))*C
    return df['score']

imdb_weighted_rating(content_based)
content_based.head()

,movieId,title,genres,numRating,avgRating,score
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,263,3.988593,3.963370
1,2,Jumanji (1995),Adventure|Children|Fantasy,107,3.289720,3.301905
2,3,Grumpier Old Men (1995),Comedy|Romance,59,3.279661,3.301784
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,8,3.750000,3.546332
4,5,Father of the Bride Part II (1995),Comedy,58,3.043103,3.106095


One hot encoding genres column

In [0]:
#onehot encoding genre at content_based df
content_based['genres'] = content_based.genres.str.split('|')

content_based.genres.apply(set)
for genres in set.union(*content_based.genres.apply(set)):
    content_based[genres] = content_based.apply(lambda x: int(x.genres.count(genres)), axis=1)

In [6]:
content_based.drop(['genres', 'movieId', 'numRating', 'avgRating'], axis=1, inplace=True)
content_based.head()

,title,score,War,Romance,Mystery,(no genres listed),Children,Thriller,Fantasy,Horror,Action,IMAX,Sci-Fi,Comedy,Animation,Musical,Western,Crime,Film-Noir,Adventure,Documentary,Drama
0,Toy Story (1995),3.963370,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0
1,Jumanji (1995),3.301905,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Grumpier Old Men (1995),3.301784,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,Waiting to Exhale (1995),3.546332,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,Father of the Bride Part II (1995),3.106095,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


## Feature Normalization

In [7]:
def normalize(df):
    result = df
    #result = df.copy()
    for feature_name in df.columns:
        if feature_name != 'title':
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

normalize(item_based)
print('Normalize item based dataframe')
normalize(content_based)
print('Normalize content based dataframe')

Normalize item based dataframe
Normalize content based dataframe


## K Nearest Neighbor Algorithm

#### Eucladian Distance

![Eucladian Distance](https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0)



In [0]:
def distance(point_p, point_q):
    #Calculate distance using Eucladian distance
    squared_difference = 0
    #calculate squared_difference between point from 1 to n
    for i in range(len(point_p)):
        squared_difference += (point_p[i] - point_q[i]) ** 2
    final_distance = squared_difference ** 0.5
    return final_distance

#### Find k Nearest Neighbor

In [0]:
def classify(title, dataset, k):
    distances = []
    title_feature = dataset[title]
    #Looping through all points in the dataset
    for data in dataset:
        point = dataset[data]
        distance_to_point = distance(title_feature, point)
        #Adding the distance and point associated with that distance
        distances.append([distance_to_point, data])
    distances.sort()
    #Taking only the k closest points
    neighbors = distances[0:k+1]
    return neighbors

## Recommendation

#### Create Dataset Dictionary from DataFrame

In [10]:
item_based_data = item_based.set_index('title').T.to_dict('list')
content_based_data = content_based.set_index('title').T.to_dict('list')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


#### Item Based Recommendation

In [11]:
title = "Avengers, The (2012)"
make_recommendation = classify(title, item_based_data, 10)
for i in range(0, len(make_recommendation)):
    if i == 0:
        print('Recommendations for {0}:\n'.format(make_recommendation[0][1]))
    else:
        print('{0}: {1}, with distance of: {2}'.format(i, make_recommendation[i][1], make_recommendation[i][0]))

Recommendations for Avengers, The (2012):

1: Dark Knight Rises, The (2012), with distance of: 3.722470970129491
2: Iron Man 3 (2013), with distance of: 3.7593176423125607
3: Super 8 (2011), with distance of: 3.7691076980508496
4: Hunger Games, The (2012), with distance of: 3.904366027164215
5: Mission: Impossible - Ghost Protocol (2011), with distance of: 3.9098640688695356
6: Captain America: The Winter Soldier (2014), with distance of: 3.919734810883808
7: Perks of Being a Wallflower, The (2012), with distance of: 3.919734810883808
8: Star Trek Into Darkness (2013), with distance of: 3.930256172343848
9: Hunger Games: Catching Fire, The (2013), with distance of: 3.9890436367383186
10: X-Men: First Class (2011), with distance of: 4.030730718301187


#### Content Based Recommendation

In [12]:
title = "Avengers, The (2012)"
make_recommendation = classify(title, content_based_data, 10)
for i in range(0, len(make_recommendation)):
    if i == 0:
        print('Recommendations for {0}:\n'.format(make_recommendation[0][1]))
    else:
        print('{0}: {1}, with distance of: {2}'.format(i, make_recommendation[i][1], make_recommendation[i][0]))

Recommendations for Avengers, The (2012):

1: Hunger Games: Catching Fire, The (2013), with distance of: 0.003603448042088786
2: Avatar (2009), with distance of: 0.014554782682697165
3: Star Trek (2009), with distance of: 0.02412667632769483
4: Spider-Man 2 (2004), with distance of: 0.06603675162232858
5: Amazing Spider-Man, The (2012), with distance of: 0.09666945845795838
6: Star Trek Into Darkness (2013), with distance of: 0.09666945845795838
7: Godzilla (2014), with distance of: 0.12538943484152287
8: John Carter (2012), with distance of: 0.13945461067987508
9: Ender's Game (2013), with distance of: 0.14626372052434733
10: Pacific Rim (2013), with distance of: 0.14987451215401393


What to do next:

*   Content based is more relevant if we had movie descriptions as feature
*   We can use another distance metric like minkowski or we can use cosine similarity

